## Intermediate SQL Topics
**Orta Düzey SQL Konuları**

Başlamadan önce, **Interactive Tutorial** eğitimini henüz tamamlamadıysan veya SQL konusunda kendini rahat hissetmiyorsan, lütfen devam etmeden önce bunu tamamla. Aşağıdaki ayrıntılı konular, gerçek dünya verileriyle uğraşırken yararlı bulabileceğin daha gelişmiş SQL yapılarını keşfederek tutorialdkai derslerin temelini oluşturacaktır.

# SQL Topic 01: Subqueries
**SQL Konu 01: Alt sorgular**

Eksiksiz bir sorguyla bile, ek posta veya ön işleme olmadan verilerimizle ilgili yanıtlayamayacağımız birçok soru olduğunu fark etmiş olabilirsin. Bu durumlarda birden çok sorgu yapıp verileri kendin işleyebilir veya SQL alt sorgularını kullanarak daha karmaşık bir sorgu oluşturabilrisin.

### Example: General subquery
**Örnek: Genel alt sorgu**

* Diyelim ki şirketin tüm satış temsilcilerinin bir listesine sahip, her iş ortağının getirdiği gelire ve bireysel maaşlarına ilişkin veriler. Zaman dar ve şimdi hangi ortaklarının şirkete, çalışan başına getirilen ortalama gelirden daha pahalıya mal olduğunu öğrenmek istiyorsun.

* Öncelikle, tüm ortakların ürettiği ortalama geliri hesaplaman gerekir:

```
SELECT AVG(revenue_generated)
FROM sales_associates;
```

* Ve sonra bu sonucu kullanarak, her bir iştirakçinin maliyetini bu değerle karşılaştırabilirsin. Bunu bir alt sorgu olarak kullanmak için, doğrudan sorgunun **WHERE** deyimine yazabilirsin: 

```
SELECT *
FROM sales_associates
WHERE salary >
    (SELECT AVG(revenue_generated)
     FROM sales_associates);
```

* Kısıtlama yürütülürken, her iştirakçinin maaşı iç alt sorgudan sorgulanan değere göre test edilecektir.

---

Bir alt sorguya, normal bir tablonun başvurulabileceği her yerde başvurulabilir. Bir **FROM** deyiminde, alt sorguları diğer tablolarla **JOIN** yapabilirsin, bir **WHERE** ya da **HAVING** ksıtılaması içinde ifadeleri alt sorgunun sonuçlarına göre test edebilirsin ve hatta doğrudan alt sorgudan veri döndürmene izin veren **SELECT** deyimindeki ifadelerde bile test edebilirsin. Son derste açıklandığı gibi, genellikle göründükleri sorgu bölümüyle aynı mantıksal sırayla yürütülürler.

Alt sorgular iç içe olabileceğinden, uygun hiyerarşi oluşturmak için her alt sorgu tamamen parantez içine alınmalıdır. Aksi takdirde, alt sorgular veritabanındaki herhangi bir tabloya başvurabilir ve normal bir sorguun yapılarını kullanabilir (ancak bazı uygulamalar alt sorguların **LIMIT** veya **OFFSET** kullanımasına izin vermez).

### Correlated subqueries
**İlişkili altsorgular**

Daha güçlü bir alt sorgu türü, iç sorgunun dış sorgudan bir kolona veya bir aliasa başvurduğu ve bunlara bağımlı olduğu ilişkili alt sorgudur. Yukarıdaki alt sorgulardan farklı olarak, iç sorgu geçerli dış sorgu satırına bağlı olduğundan, bu iç sorguların her birinin dış sorgudaki satıların her biri için çalıştırılması gerekir.

#### Örnek

Yukarıdaki satış ortakları listesi yerine, çalışanların, departmalarının (mühendislik, satıi vb.), gelirlerinin ve maaşlarının genel bir listesine sahip olduğunu hayal et. Bu sefer, departmanlarında ortalamanın üzerinde performans gösteren çalışanları bulmak için şirket genelinde bakıyorsun. 

Her çalışan için, departmanlarındaki tüm insanlar tarafından üretilen ortalama gelire göre maliyetlerini hesaplaman gerekir. Departmanın ortalamasını almak için, alt sorgunun her çalışanın hangi departmanda olduğunu bilmesi gerekir:

```
SELECT *
FROM employees
WHERE salary >
    (SELECT AVG(revenue_generated)
     FROM employees AS dept_employees
     WHERE dept_employees.department = employees.department);
```

Bu tür karmaşık soruglar güçlü olabilir, ancak aynı zamanda okunması ve anlaşılması zor olabilir, bu nedenle bunları kullanırken dikkatli olmalısın. Mümkünse, geçici değerlere ve tablolara anlamlı aliaslar vermeyi dene. Ek olarak, ilişkili alt sorguların optimize edilmesi zor olabilir, bu nedenle performans özellikleri farlı veritabanlarında değişiklik gösterebilir.

### Existence tests

**WHERE** kısıtlamaları tanıtılırken, mevcut satırdaki kolon değerinin sabit bir değerler listesinde var olup olmadığını test etkmek için **IN** operatörü kullanıldı. Karmaşık sorgularda bu, dinamik bir değerler listesinde bir kolon değerinin var olup olmadığını test etmek için alt sorgular kullanılarak genişletilebilir.

```
SELECT *, ...
FROM mytable
WHERE column
    IN/NOT IN (SELECT another_column
               FROM another_table)
```

Bunu yaparken, dış kolonun test edilebileceği bir liste oluşturmak için alt sorgunun bir sütun değeri veya ifadesi seçmesi gerektiğine dikkat et. Bu kısıtlama türü, kısıtlamalar mevcut verilere dayandığında güçlüdür.